In [1]:
#' uniRank University Scraper
#'
#' Downloads University Revie data from uniRank (https://www.4icu.org/reviews/*.htm)
#' and saves it into a CSV
#'
#' Author: Jacob J. Walker
#' 
#' Last Published for Automation: 
import time; section_start = {'Full Script': time.time()}

In [2]:
# Set Script General Parameters                                                                        # Code Block Info: 2020-06-11 From Python TemplateG

DEFAULT_SLEEP = 5                          # Number of seconds that should be paused between actions that need pausing
TIMEOUT_MINUTES = 60                       # Variable used to Know when to Timeout
DRY_RUN = False                            # Variables used for Testing

In [3]:
# Set Path and Filename Parameters                                                      # Code Block Info: 2020-06-11 From Python Template

SAVE_PATH = "C:/Users/Jacob.Walker/Downloads"
SAVE_FILE_BASE = "uniRank"
SAVE_FILE_EXT = ".csv"

In [4]:
###########################
section =  'Initialization' 
###########################
import time; section_start[section] = time.time()

In [5]:
# Initialize Package Loader Function                                                                  # Code Block Info: 2020-04-27 From Python Template
# Acts similar to R's pacman p_load

# Modules used in Code Block
import pip
import importlib


def p_load(package, *functions_, as_=None, module=None):
    if as_ is None:
        as_ = package
    
    if module is None:
        module = package
        
    try:
        globals()[as_] = importlib.import_module(module)
    except:
        if hasattr(pip, 'main'):
            pip.main(['install', package])
        else:
            pip._internal.main(['install', package])
        globals()[as_] = importlib.import_module(package)
        
# Need to figure out how to load individual functions
# See https://stackoverflow.com/questions/56902954/how-to-use-from-x-import-y-using-importlib-in-python
# See https://www.informit.com/articles/article.aspx?p=2314818          

In [6]:
# Set Path and Filename Variables based on original parameters                           # Code Block Info: 2020-06-12 From Python Template

# Modules used in Code Block
p_load('pandas', as_='pd')          # Package that provides dataframes similar to R as well as a lot of other data manipulation abilities
p_load('arrow')                     # Similar to lubridate, although it has a different syntax
p_load('re')                        # Includes sub to act like gsub

# Get dates for date stamps
date_stamp = arrow.now('US/Pacific').strftime("%Y-%m-%d %H%M")
doc_date = arrow.now('US/Pacific').strftime("%m/%d/%Y %I:%M %p")

# Convert backslashes to forward slashes to be more Python friendly
SAVE_PATH = re.sub("\\\\", "/", SAVE_PATH)

# Set a full save path that includes the date/time stamp
FULL_SAVE_PATH = SAVE_PATH + "/" + SAVE_FILE_BASE + " " + date_stamp + SAVE_FILE_EXT

# Set Pandas Options
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)  

In [7]:
# Set Up Logging                                                                                 # Code Block Info: 2020-04-30 From Python Template

# Modules used in Code Block
p_load('loguru')                    # Package to make python logging simple
p_load('warnings')                  # Built-in module to deal with warnings
p_load('urllib')                    # Package to handle working with URLs and web pages
p_load('json')                      # Packge to handle json
p_load('os')                        # Built-in module similar to R's sys
p_load('sys')                       # Built-in module similar to R's sys
p_load('notebook')                  # Package used for Jupyter Notebooks
p_load('ipykernel')                 # Another Package for Jupyter
from notebook import notebookapp
from loguru import logger


# Functions Used in Code Block

def notebook_path():
    # See https://stackoverflow.com/questions/12544056/how-do-i-get-the-current-ipython-jupyter-notebook-name
    """Returns the absolute path of the Notebook or None if it cannot be determined
    NOTE: works only when the security is token-based or there is also no password
    """
    connection_file = os.path.basename(ipykernel.get_connection_file())
    kernel_id = connection_file.split('-', 1)[1].split('.')[0]

    for srv in notebookapp.list_running_servers():
        try:
            if srv['token']=='' and not srv['password']:  # No token and no password, ahem...
                req = urllib.request.urlopen(srv['url']+'api/sessions')
            else:
                req = urllib.request.urlopen(srv['url']+'api/sessions?token='+srv['token'])
            sessions = json.load(req)
            for sess in sessions:
                if sess['kernel']['id'] == kernel_id:
                    return os.path.join(srv['notebook_dir'],sess['notebook']['path'])
        except:
            pass  # There may be stale entries in the runtime directory 
    return None

def script_name():
    try:
        return os.path.splitext(os.path.basename(notebook_path()))[0]
    except:
        return os.path.splitext(os.path.basename(__file__))[0]

def script_path():
    try:
        return os.path.dirname(notebook_path())
    except:
        return os.path.dirname(os.path.realpath(__file__))  
    
    
# Logs all Warnings Emitted by Application
showwarning_ = warnings.showwarning

def showwarning(message, *args, **kwargs):
    logger.warning(message)
    showwarning_(message, *args, **kwargs)

warnings.showwarning = showwarning

# Logs all Exceptions (Only works with .py files, not in Jupyter)
excepthook_ = sys.excepthook

def excepthook(type, value, traceback):
    logger.exception(type + ": " + value)
    excepthook_(type, value, traceback)

sys.excepthook = excepthook


# Code Block    
print(script_name())
logger.remove()
logger.add(sys.stderr, level="INFO")
logger.add(script_path()+"/"+script_name()+".log",  backtrace=False, level="SUCCESS")

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Scraper - uniRank


2

In [8]:
#################################################################################################################################################
logger.success(section + " completed in " + time.strftime("%H hours %M minutes and %S seconds", time.gmtime(time.time()-section_start[section])))
#################################################################################################################################################

2020-06-20 14:25:03.369 | SUCCESS  | __main__:<module>:2 - Initialization completed in 00 hours 00 minutes and 11 seconds


In [9]:
##########################
section = 'Web Scraping'
##########################
import time; section_start[section] = time.time()

In [19]:
# Run Selenium                                                                                      # Code Block Info: 2020-06-20 From Python Template

GECKO_DRIVER_PATH = 'C:/Users/Jacob.Walker/AppData/Local/binman/binman_geckodriver/win64/0.26.0'


# Modules used in Code Block
p_load('selenium')                  # Similar to RSelenium package
p_load('re')                        # Includes sub to act like gsub
p_load('os')
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Ensures Path points to Gecko Driver Folder
orig_path = os.environ.get('PATH')
os.environ['PATH'] = orig_path + ';' + GECKO_DRIVER_PATH

# Set FireFox Settings
fp = webdriver.FirefoxProfile()

AutoSaveMIMETypes = "text/csv, application/csv, charset=utf-8, text/comma-separated-values, text/plain, application/vnd.ms-excel, application/octet-stream, application/download"
fp.set_preference("browser.helperApps.neverAsk.saveToDisk",AutoSaveMIMETypes)
fp.set_preference("browser.download.manager.showWhenStarting",False)
fp.set_preference("browser.download.dir",re.sub("/", "\\\\", SAVE_PATH))
fp.set_preference("browser.download.folderList",2)
fp.set_preference("browser.helperApps.alwaysAsk.force",False)
Enter = "\uE003"

# Start the Selenium web driver
web_driver = webdriver.Firefox(firefox_profile=fp)
default_page_load_timeout = web_driver.capabilities['timeouts']["pageLoad"]

In [24]:
# Create Dictionary of fields and source

dict = { 'name':                 "/html/body/div[3]/div[1]/div/div/div[2]/h1"
       , 'url':                  "/html/body/div[3]/div[3]/div[1]/div/div[2]/table/tbody/tr[1]/td/a"
       , 'alternateName':        "/html/body/div[3]/div[3]/div[1]/div/div[2]/table/tbody/tr[2]/td/a/span"
       , 'abbr':                 "/html/body/div[3]/div[3]/div[1]/div/div[2]/table/tbody/tr[3]/td/abbr"
       , 'countryRank':          "/html/body/div[3]/div[1]/div/div/div[3]/table/tbody/tr[1]/td[2]/a/strong"
       , 'worldRank':            "/html/body/div[3]/div[1]/div/div/div[3]/table/tbody/tr[2]/td[2]/a/strong"
       , 'description':          "/html/body/div[3]/div[2]/div[2]/div/div[2]/p"
       , 'foundingDate':         "/html/body/div[3]/div[3]/div[1]/div/div[2]/table/tbody/tr[4]/td/span"
       , 'founded':              "/html/body/div[3]/div[3]/div[1]/div/div[2]/table/tbody/tr[4]/td"
       , 'streetAddress':        "/html/body/div[3]/div[3]/div[2]/div[2]/div[2]/table/tbody/tr[1]/td/span[1]"
       , 'addressLocality':      "/html/body/div[3]/div[3]/div[2]/div[2]/div[2]/table/tbody/tr[1]/td/span[2]"
       , 'postalCode':           "/html/body/div[3]/div[3]/div[2]/div[2]/div[2]/table/tbody/tr[1]/td/span[3]"
       , 'addressRegion':        "/html/body/div[3]/div[3]/div[2]/div[2]/div[2]/table/tbody/tr[1]/td/span[4]"
       , 'address':              "/html/body/div[3]/div[3]/div[2]/div[2]/div[2]/table/tbody/tr[1]/td"
       , 'telephone':            "/html/body/div[3]/div[3]/div[2]/div[2]/div[2]/table/tbody/tr[2]/td/span"
       , 'faxNumber':            "/html/body/div[3]/div[3]/div[2]/div[2]/div[2]/table/tbody/tr[3]/td/span"
       , 'diploma':              "/html/body/div[3]/div[4]/div[2]/div[1]/table/thead/tr[3]/td[1]/i"
       , 'bachelor':             "/html/body/div[3]/div[4]/div[2]/div[1]/table/thead/tr[3]/td[2]/i"
       , 'master':               "/html/body/div[3]/div[4]/div[2]/div[1]/table/thead/tr[3]/td[3]/i"
       , 'doctorate':            "/html/body/div[3]/div[4]/div[2]/div[1]/table/thead/tr[3]/td[4]/i"
       , 'diplomaArts':          "/html/body/div[3]/div[4]/div[2]/div[1]/table/tbody/tr[1]/td[2]/i"
       , 'bachelorArts':         "/html/body/div[3]/div[4]/div[2]/div[1]/table/tbody/tr[1]/td[3]/i"
       , 'masterArts':           "/html/body/div[3]/div[4]/div[2]/div[1]/table/tbody/tr[1]/td[4]/i"
       , 'doctorateArts':        "/html/body/div[3]/div[4]/div[2]/div[1]/table/tbody/tr[1]/td[5]/i"
       , 'diplomaBusiness':      "/html/body/div[3]/div[4]/div[2]/div[1]/table/tbody/tr[2]/td[2]/i"
       , 'bachelorBusiness':     "/html/body/div[3]/div[4]/div[2]/div[1]/table/tbody/tr[2]/td[3]/i"
       , 'masterBusiness':       "/html/body/div[3]/div[4]/div[2]/div[1]/table/tbody/tr[2]/td[4]/i"
       , 'doctorateBusiness':    "/html/body/div[3]/div[4]/div[2]/div[1]/table/tbody/tr[2]/td[5]/i"
       , 'diplomaLanguage':      "/html/body/div[3]/div[4]/div[2]/div[1]/table/tbody/tr[3]/td[2]/i"
       , 'bachelorLanguage':     "/html/body/div[3]/div[4]/div[2]/div[1]/table/tbody/tr[3]/td[3]/i"
       , 'masterLanguage':       "/html/body/div[3]/div[4]/div[2]/div[1]/table/tbody/tr[3]/td[4]/i"
       , 'doctorateLanguage':    "/html/body/div[3]/div[4]/div[2]/div[1]/table/tbody/tr[3]/td[5]/i"
       , 'diplomaMedicine':      "/html/body/div[3]/div[4]/div[2]/div[1]/table/tbody/tr[4]/td[2]/i"
       , 'bachelorMedicine':     "/html/body/div[3]/div[4]/div[2]/div[1]/table/tbody/tr[4]/td[3]/i"
       , 'masterMedicine':       "/html/body/div[3]/div[4]/div[2]/div[1]/table/tbody/tr[4]/td[4]/i"
       , 'doctorateMedicine':    "/html/body/div[3]/div[4]/div[2]/div[1]/table/tbody/tr[4]/td[5]/i"
       , 'diplomaEngineering':   "/html/body/div[3]/div[4]/div[2]/div[1]/table/tbody/tr[5]/td[2]/i"
       , 'bachelorEngineering':  "/html/body/div[3]/div[4]/div[2]/div[1]/table/tbody/tr[5]/td[3]/i"
       , 'masterEngineering':    "/html/body/div[3]/div[4]/div[2]/div[1]/table/tbody/tr[5]/td[4]/i"
       , 'doctorateEngineering': "/html/body/div[3]/div[4]/div[2]/div[1]/table/tbody/tr[5]/td[5]/i"
       , 'diplomaScience':       "/html/body/div[3]/div[4]/div[2]/div[1]/table/tbody/tr[6]/td[2]/i"
       , 'bachelorScience':      "/html/body/div[3]/div[4]/div[2]/div[1]/table/tbody/tr[6]/td[3]/i"
       , 'masterScience':        "/html/body/div[3]/div[4]/div[2]/div[1]/table/tbody/tr[6]/td[4]/i"
       , 'doctorateScience':     "/html/body/div[3]/div[4]/div[2]/div[1]/table/tbody/tr[6]/td[5]/i"
       , 'tuitionLocalUndergrad':"/html/body/div[3]/div[5]/div[2]/div/table/tbody/tr[1]/td[2]"
       , 'tuitionLocalPostgrad': "/html/body/div[3]/div[5]/div[2]/div/table/tbody/tr[1]/td[3]"
       , 'tuitionIntUndergrad':  "/html/body/div[3]/div[5]/div[2]/div/table/tbody/tr[2]/td[2]"
       , 'tuitionIntPostgrad':   "/html/body/div[3]/div[5]/div[2]/div/table/tbody/tr[2]/td[3]"
       , 'gender':               "/html/body/div[3]/div[6]/div[2]/div/div[2]/table/tbody/tr[1]/td"
       , 'internationalStudents':"/html/body/div[3]/div[6]/div[2]/div/div[2]/table/tbody/tr[2]/td"
       , 'selectionType':        "/html/body/div[3]/div[6]/div[2]/div/div[2]/table/tbody/tr[3]/td"
       , 'admissionRate':        "/html/body/div[3]/div[6]/div[2]/div/div[2]/table/tbody/tr[4]/td"
       , 'admissionOffice':      "/html/body/div[3]/div[6]/div[2]/div/div[2]/table/tbody/tr[5]/td"
       , 'studentEnrollment':    "/html/body/div[3]/div[7]/div[2]/div/div[1]/table/tbody/tr[1]/td"
       , 'academicCalendar':     "/html/body/div[3]/div[7]/div[2]/div/div[2]/table/tbody/tr[1]/td"
       , 'academicStaff':        "/html/body/div[3]/div[7]/div[2]/div/div[1]/table/tbody/tr[2]/td"
       , 'campusSetting':        "/html/body/div[3]/div[7]/div[2]/div/div[2]/table/tbody/tr[2]/td"
       , 'controlType':          "/html/body/div[3]/div[7]/div[2]/div/div[1]/table/tbody/tr[3]/td"
       , 'religiousAffiliation': "/html/body/div[3]/div[7]/div[2]/div/div[2]/table/tbody/tr[3]/td"
       , 'entityType':           "/html/body/div[3]/div[7]/div[2]/div/div[1]/table/tbody/tr[4]/td"
       , 'library':              "/html/body/div[3]/div[8]/div[2]/div/div[1]/table/tbody/tr[1]/td"
       , 'financialAids':        "/html/body/div[3]/div[8]/div[2]/div/div[2]/table/tbody/tr[1]/td"
       , 'housing':              "/html/body/div[3]/div[8]/div[2]/div/div[1]/table/tbody/tr[2]/td"
       , 'studyAbroad':          "/html/body/div[3]/div[8]/div[2]/div/div[2]/table/tbody/tr[2]/td"
       , 'sportFacilities':      "/html/body/div[3]/div[8]/div[2]/div/div[1]/table/tbody/tr[3]/td"
       , 'distanceLearning':     "/html/body/div[3]/div[8]/div[2]/div/div[2]/table/tbody/tr[3]/td"
       , 'accreditation':        "/html/body/div[3]/div[9]/div[2]/p[1]/a"
       , 'accreditation_url':    "/html/body/div[3]/div[9]/div[2]/p[1]/a"
       , 'yearOfFirstAccredit':  "/html/body/div[3]/div[9]/div[2]/p[2]"
       , 'facebook_url':         "/html/body/div[3]/div[12]/div[2]/p[1]/a"
       , 'twitter_url':          "/html/body/div[3]/div[12]/div[2]/p[2]/a"
       , 'linkedIn_url':         "/html/body/div[3]/div[12]/div[2]/p[3]/a"
       , 'youTube_url':          "/html/body/div[3]/div[12]/div[2]/p[4]/a"
       , 'instagram_url':        "/html/body/div[3]/div[12]/div[2]/p[5]/a"
       , 'wikipedia_url':        "/html/body/div[3]/div[14]/div[2]/p/a"}

In [25]:
# Create an Empty Dataframe for the web pages

# Modules used in Code Block
p_load('pandas', as_='pd')          # Package that provides dataframes similar to R as well as a lot of other data manipulation abilities

df = pd.DataFrame(columns=dict.keys())

In [26]:
# Go to uniRank                                                                                     # Code Block Info: 2020-06-04 From Python Template

# Modules used in Code Block
p_load('selenium')                  # Similar to RSelenium package
p_load('keyring')                   # Similar to R's keyring package
p_load('time')                      # Built-in module that will allow sleep (pausing for a few seconds)
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

for i in range(1,100000):
    web_driver.get("https://www.4icu.org/reviews/"+str(i)+".htm")
    df.loc[i] = ''
    df.loc[i]['index'] = i
    for column in dict.keys():
        try:
            web_elem = web_driver.find_element_by_xpath(dict[column])
            if column[-3:] == 'url':
                df.loc[i][column] = web_elem.get_attribute('href')
            elif web_elem.text == '':
                df.loc[i][column] = web_elem.get_attribute('class')
            else:
                df.loc[i][column] = web_elem.text
        except:
            pass
   


InvalidSessionIdException: Message: Tried to run command without establishing a connection


In [27]:
df

,name,url,alternateName,abbr,countryRank,worldRank,description,foundingDate,founded,streetAddress,addressLocality,postalCode,addressRegion,address,telephone,faxNumber,diploma,bachelor,master,doctorate,diplomaArts,bachelorArts,masterArts,doctorateArts,diplomaBusiness,bachelorBusiness,masterBusiness,doctorateBusiness,diplomaLanguage,bachelorLanguage,masterLanguage,doctorateLanguage,diplomaMedicine,bachelorMedicine,masterMedicine,doctorateMedicine,diplomaEngineering,bachelorEngineering,masterEngineering,doctorateEngineering,diplomaScience,bachelorScience,masterScience,doctorateScience,tuitionLocalUndergrad,tuitionLocalPostgrad,tuitionIntUndergrad,tuitionIntPostgrad,gender,internationalStudents,selectionType,admissionRate,admissionOffice,studentEnrollment,academicCalendar,academicStaff,campusSetting,controlType,religiousAffiliation,entityType,library,financialAids,housing,studyAbroad,sportFacilities,distanceLearning,accreditation,accreditation_url,yearOfFirstAccredit,facebook_url,twitter_url,linkedIn_url,youTube_url,instagram_url,wikipedia_url
1,Universiteti Politeknik i Tiranës,http://www.upt.al/,Polytechnic University of Tirana,UPT,4,8834,"Founded in 1991, Universiteti Politeknik i Tiranës (Polytechnic University of Tirana) is a non-profit public higher education institution located in the urban setting of the medium-sized city of Tirana (population range of 500,000-1,000,000 inhabitants). Officially accredited and/or recognized by the Agjencia e Sigurimit të Cilësisë në Arsimin e Lartë (Quality Assurance Agency in Higher Education), Universiteti Politeknik i Tiranës (UPT) is a large (uniRank enrollment range: 10,000-14,999 students) coeducational higher education institution. Universiteti Politeknik i Tiranës (UPT) offers courses and programs leading to officially recognized higher education degrees such as bachelor degrees, master degrees, doctorate degrees in several areas of study. See the uniRank degree levels and areas of study matrix below for further details. This 29 years old higher-education institution has a selective admission policy based on students' past academic record and grades.",1991,1991 Origins date back to 1951.,"Sheshi Nënë Tereza, 4","Tirana (population range: 500,000-1,000,000)",1000,Tirana,"Sheshi Nënë Tereza, 4\nTirana (population range: 500,000-1,000,000)\n1000 Tirana\nAlbania",+355 (4) 226 9218,+355 (4) 224 0504,sp sp-0,sp sp-1,sp sp-1,sp sp-1,sp sp-0,sp sp-0,sp sp-0,sp sp-0,sp sp-0,sp sp-0,sp sp-0,sp sp-0,sp sp-0,sp sp-0,sp sp-0,sp sp-0,sp sp-0,sp sp-0,sp sp-0,sp sp-0,sp sp-0,sp sp-1,sp sp-1,sp sp-1,sp sp-0,sp sp-1,sp sp-1,sp sp-1,"1,000-2,500 US$\n(750-1,800 Euro)",Not reported,Not reported,Not reported,Men and Women (coed),"Yes, international applicants are welcome to apply for admission","Yes, based on students' past academic record and grades",Not reported,+355,"10,000-14,999",Semesters,700-799,Urban,Public,None,Non-Profit,Yes,Yes,Not reported,Yes,Not reported,Not reported,Agjencia e Sigurimit të Cilësisë në Arsimin e Lartë,https://www.4icu.org/institutions/al/531.htm,1951,,,https://www.linkedin.com/school/universiteti-politeknik-i-tiran%C3%ABs/,,,https://en.wikipedia.org/wiki/Polytechnic_University_of_Tirana
2,Universiteti i Elbasanit Aleksander Xhuvani,http://www.uniel.edu.al/,Aleksander Xhuvani University of Elbasan,,5,9288,"Established in 1971, Universiteti i Elbasanit Aleksander Xhuvani (Aleksander Xhuvani University of Elbasan) is a non-profit public higher education institution located in the large town of Elbasan (population range of 50,000-249,999 inhabitants). Officially accredited and/or recognized by the Agjencia e Sigurimit të Cilësisë në Arsimin e Lartë (Quality Assurance Agency in Higher Education), Universiteti i Elbasanit Aleksander Xhuvani is a coeducational higher education institution. Universiteti i Elbasanit Aleksander Xhuvani offers courses and programs leading to officially recognized higher education degrees such as bachelor degrees in several areas of study. See the uni